In [72]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from random import randint
import random

In [2]:
ROWS      = 5
COLUMNS   = 7
WALL_SIZE = 7
ACTIONS   = 4
STATES    = ROWS*COLUMNS 
γ         = 0.9  # γάμμα

# This is my world
M =([0,  0,  0, 0, 0,  0, 0],
    [0,  1, -1, 0, 0,  0, 0],
    [0, -1, -1, 0, 0,  0, 0],
    [0,  0, -1, 0, 0, -1, 0],
    [0,  0,  0, 0, 1, -1, 2])

# Actions
A=["E", "W", "N","S"] # East, West, North, South

# States
S = ((0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6),
     (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
     (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6),
     (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6),
     (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6),
     )

# Start and Goal
starting_state = S[8]
goal_state     = S[34]

# Wall 
W=((1, 2), (2, 1), (2, 2), (2, 1), (3, 2), (3, 5), (4, 5))

In [84]:
def ConvertIndexToTuple(state):
    return(tuple(int(x) for x in np.base_repr(state, 7, 1)[-2::]))

def ConvertTupleToIndex(state):
    return(int("".join(str(x) for x in state), 7))

def getNextState(state, action):
    # Check if we reached goal state
    if state == goal_state:
        return state
    
    if action == "E":
        next_state_index=ConvertTupleToIndex(state)+1
    elif action == "W":
        next_state_index=ConvertTupleToIndex(state)-1 
    elif action == "N":
        next_state_index=ConvertTupleToIndex(state)-COLUMNS
    elif action == "S":
        next_state_index=ConvertTupleToIndex(state)+COLUMNS 
             
    # Check if next state hits the wall.
    for i in range(0,WALL_SIZE):
        if next_state_index == ConvertTupleToIndex(W[i]):
            return state
            
    # Check if next state is within Maze.
    if next_state_index >=0 and next_state_index <=34:
        next_state=ConvertIndexToTuple(next_state_index)
        return next_state
    else:
        return state    
    
#convertIndexToTuple(34)    
#convertTupleToIndex(starting_state)
#getNextState(starting_state, A[3])
print(S[6], A[0])
getNextState(S[6], A[0])

(0, 6) E


(1, 0)

In [86]:
def StateTransition(current_state, action, next_state):
    #next_state = getNextState( current_state, action) 
    if current_state == next_state:
        return 1
    else:
        return 0.7
    
def getReward(current_state, action, next_state):
    if next_state == goal_state:
        return 10
    else:
        return 0

# Might keep it.
def weighted_choice(weights):
    totals = []
    running_total = 0

    for w in weights:
        running_total += w
        totals.append(running_total)

    rnd = random.random() * running_total
    for i, total in enumerate(totals):
        if rnd < total:
            return i    
# for i in range (0,100):        
#     print(weighted_choice([7,1,1,1]))        

In [90]:
# Define P and R matrices.
P = [[0 for x in range(ACTIONS)] for y in range(STATES)]  # Probabilty
R = [[0 for x in range(ACTIONS)] for y in range(STATES)]  # Reward
N = [[0 for x in range(ACTIONS)] for y in range(STATES)]  # Next State

# Initialize P, R and N
for i in range (0, STATES):
    for j in range (0, ACTIONS):
        current_state = S[i]
        action = A[j]
        next_state = getNextState(current_state, action)
        N[i][j] = next_state
        P[i][j] = StateTransition(current_state, action, next_state)
        R[i][j] = getReward(current_state, action, next_state) 
        
N        

[[(0, 1), (0, 0), (0, 0), (1, 0)],
 [(0, 2), (0,), (0, 1), (1, 1)],
 [(0, 3), (0, 1), (0, 2), (0, 2)],
 [(0, 4), (0, 2), (0, 3), (1, 3)],
 [(0, 5), (0, 3), (0, 4), (1, 4)],
 [(0, 6), (0, 4), (0, 5), (1, 5)],
 [(1, 0), (0, 5), (0, 6), (1, 6)],
 [(1, 1), (0, 6), (0,), (2, 0)],
 [(1, 1), (1, 0), (0, 1), (1, 1)],
 [(1, 3), (1, 1), (0, 2), (1, 2)],
 [(1, 4), (1, 3), (0, 3), (2, 3)],
 [(1, 5), (1, 3), (0, 4), (2, 4)],
 [(1, 6), (1, 4), (0, 5), (2, 5)],
 [(2, 0), (1, 5), (0, 6), (2, 6)],
 [(2, 0), (1, 6), (1, 0), (3, 0)],
 [(2, 1), (2, 0), (1, 1), (3, 1)],
 [(2, 3), (2, 2), (2, 2), (2, 2)],
 [(2, 4), (2, 3), (1, 3), (3, 3)],
 [(2, 5), (2, 3), (1, 4), (3, 4)],
 [(2, 6), (2, 4), (1, 5), (2, 5)],
 [(3, 0), (2, 5), (1, 6), (3, 6)],
 [(3, 1), (2, 6), (2, 0), (4, 0)],
 [(3, 1), (3, 0), (3, 1), (4, 1)],
 [(3, 3), (3, 1), (3, 2), (4, 2)],
 [(3, 4), (3, 3), (2, 3), (4, 3)],
 [(3, 4), (3, 3), (2, 4), (4, 4)],
 [(3, 6), (3, 4), (2, 5), (3, 5)],
 [(4, 0), (3, 6), (2, 6), (4, 6)],
 [(4, 1), (3, 6), (3, 0)

In [79]:
# current_state  = starting_state # S[8]

# Initialize V(s)=0
V = [0 for x in range(STATES)] 
Q = [[0 for x in range(ACTIONS)] for y in range(STATES)] 

# Initialize π(s,α)=0.25
π = [[0.25 for x in range(ACTIONS)] for y in range(STATES)]

action = A[randint(0,3)] # Random Policy

# next_state = getNextState(current_state, action)
# StateTransition(current_state, action, next_state)
# getReward(starting_state, action, next_state )

0

In [49]:
# Ex 1

current_state  = starting_state
action = A[0]

next_state = getNextState(current_state, action)
prob_next  = StateTransition(current_state, action, next_state)
reward     = getReward(current_state, action, next_state)

print ('Current state : ' , current_state)
print ('Move to       : ' , action)
print ('Next state    : ' , next_state)
print ('Probability of Next State : ', prob_next )
print ('Reward : ', reward )

Current state :  (1, 1)
Move to       :  E
Next state    :  (1, 1)
Probability of Next State :  1
Reward :  0


In [50]:
# Ex 2

current_state  = starting_state
counter = 0
print ('Initial state : ' , current_state)
#while current_state != goal_state:
for i in range (0, 1000):
    action = A[randint(0,3)]
    next_state = getNextState(current_state, action)
    prob_next  = StateTransition(current_state, action, next_state)
    reward     += getReward(current_state, action, next_state)
    counter= counter+1
    print (current_state)
    print ()
    current_state = next_state
    if current_state == goal_state:
        print ('Goal Reached! : ' , current_state)
        break
print ('Current state : ' , current_state)
print ('Move to       : ' , action)
print ('Next state    : ' , next_state)
print(counter)   

Initial state :  (1, 1)
Goal Reached! :  (4, 6)
Current state :  (4, 6)
Move to       :  S
Next state    :  (4, 6)
290
